In [26]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Data

In [27]:
train = pd.read_csv('tabular-playground-series-feb-2022/train.csv', index_col='row_id')
test = pd.read_csv('tabular-playground-series-feb-2022/test.csv', index_col='row_id')
train.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
row_id,,,,,,,,,,,,,,,,,,,,,
0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [28]:
duplicates_train = train.duplicated().sum()
print('Duplicates in train data: {0}'.format(duplicates_train))

train.drop_duplicates(keep='first', inplace=True)
duplicates_train = train.duplicated().sum()

print('Train data shape:', train.shape)
print('Duplicates in train data: {0}'.format(duplicates_train))

Duplicates in train data: 76007
Train data shape: (123993, 287)
Duplicates in train data: 0


In [30]:
# Pseudo-labeling!
pseudo_labels = pd.read_csv('pseudo_labels.csv')
print(pseudo_labels.shape)
pseudo_labels.head()

(100000, 2)


,row_id,target
0,200000,Escherichia_fergusonii
1,200001,Salmonella_enterica
2,200002,Enterococcus_hirae
3,200003,Salmonella_enterica
4,200004,Staphylococcus_aureus


In [31]:
pseudo_train = X_test.copy()
pseudo_train['target'] = pseudo_labels['target'].values
train = pd.concat([train, pseudo_train]).reset_index(drop=True)
print(train.shape)
train.head()

(223993, 287)


,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [32]:
X = data.drop(['target'], axis=1)
y = data.target

In [33]:
le = LabelEncoder()
y = le.fit_transform(y)

# Hyperparameter Optimization

In [39]:
%env WANDB_NOTEBOOK_NAME xgboost.ipynb

env: WANDB_NOTEBOOK_NAME=xgboost.ipynb


In [40]:
import wandb
wandb.login()

True

In [41]:
sweep_config = {
    "method": "random", # try grid or random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree","gblinear"]
        },
        "max_depth": {
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.05, 0.2]
        },
        "subsample": {
            "values": [1, 0.5, 0.3]
        }
    }
}

In [57]:
sweep_id = wandb.sweep(sweep_config, project="tab-playground-feb-2022")

Create sweep with ID: j2awpog5
Sweep URL: https://wandb.ai/eddiezhuang/tab-playground-feb-2022/sweeps/j2awpog5


In [56]:
 X_train, X_test, y_train, y_test = train_test_split(X, y)

# fit model on train
model = XGBClassifier(n_jobs=6, use_label_encoder=False)
model.fit(X_train, y_train)

# make predictions on test
y_pred = model.predict(X_test)
print(y_pred)

[17:42:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[3 2 5 ... 7 8 6]


In [58]:
def training():
    config_defaults = {
        "booster": "gbtree",
        "max_depth": 3,
        "learning_rate": 0.1,
        "subsample": 1,
        "seed": 117,
        "test_size": 0.33,
    }

    wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
    config = wandb.config

    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=config.test_size,
                                                      random_state=config.seed)

    # fit model on train
    model = XGBClassifier(booster=config.booster, max_depth=config.max_depth,
                          learning_rate=config.learning_rate, subsample=config.subsample, 
                          n_jobs=6, use_label_encoder=False)
    model.fit(X_train, y_train)

    # make predictions on test
    y_pred = model.predict(X_test)

    # evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {int(accuracy * 100.)}%")
    wandb.log({"accuracy": accuracy})

In [ ]:
wandb.agent(sweep_id, training, count=25)

wandb: Agent Starting Run: 505afipf with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 6
wandb: 	subsample: 1


Accuracy: 91%


accuracy,▁
accuracy,0.91133


wandb: Agent Starting Run: vd4ymq2y with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 3
wandb: 	subsample: 0.3


D:\Users\zhuan\anaconda3\envs\ml\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
best_config = {
    "booster": "gbtree",
    "max_depth": 3,
    "learning_rate": 0.1,
    "subsample": 1,
}

# Inference

In [ ]:
model = XGBClassifier(booster=best_config.booster, max_depth=best_config.max_depth,
                        learning_rate=best_config.learning_rate, subsample=best_config.subsample)

In [16]:
predictions = model.predict(test)

D:\Users\zhuan\anaconda3\envs\ml\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [21]:
preds = le.inverse_transform(np.argmax(predictions, axis=1))

output = pd.DataFrame({'row_id': X_test.index,
                       'target': preds})

output.to_csv('submission.csv', index=False)

output.head()

,row_id,target
0,200000,Escherichia_fergusonii
1,200001,Salmonella_enterica
2,200002,Enterococcus_hirae
3,200003,Salmonella_enterica
4,200004,Staphylococcus_aureus


In [22]:
!kaggle competitions submit -c tabular-playground-series-feb-2022 -f submission.csv -m ""

Successfully submitted to Tabular Playground Series - Feb 2022



  0%|          | 0.00/2.81M [00:00<?, ?B/s]
  3%|2         | 80.0k/2.81M [00:00<00:03, 755kB/s]
 26%|##6       | 752k/2.81M [00:00<00:00, 4.21MB/s]
 41%|####      | 1.15M/2.81M [00:00<00:00, 2.32MB/s]
 56%|#####5    | 1.57M/2.81M [00:00<00:00, 1.64MB/s]
 70%|#######   | 1.98M/2.81M [00:00<00:00, 2.10MB/s]
 81%|########  | 2.27M/2.81M [00:01<00:00, 2.24MB/s]
 91%|######### | 2.55M/2.81M [00:01<00:00, 1.44MB/s]
 98%|#########8| 2.76M/2.81M [00:01<00:00, 1.55MB/s]
100%|##########| 2.81M/2.81M [00:02<00:00, 1.20MB/s]
